In [1]:
# website scraping

In [2]:
import requests
from bs4 import BeautifulSoup
import dateparser
import re
import pandas as pd
import numpy as np

to do list:


-~~ if contains class="more-link", redirect the link and use the link to update the post~~ 

- factorize the functions, and use a class to 
- search for report date based on the post date + day of the week (more flexible)
- improve the doc-string quality
- find a way to split the wetsuit info for shoe, wetsuit,glove and hoed

In [3]:
def read_posts(url):
    """
    extract all the posts on each webpage
    """
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    posts = soup.find_all(id=re.compile("post"))

    return posts


def forecast_parser(post):
    """
    loop through each paragrah using the headings contained in the tag <strong>
    """
    tags = post.find_all('strong')

    dates = []
    contents = []
    days_of_week = []

    for tag in tags:
        # dates needs to be corrected using the year specified in the post
        date = dateparser.parse(tag.get_text())
        dayofweek = tag.get_text().split()[0].lower()
        dates.append(date)
        days_of_week.append(dayofweek)

        content = tag.parent.get_text()
        contents.append(content)

    return dates, contents, days_of_week


def post_to_df(post):
    """
    extract data from each post, write to dataframe 
    """
    post_id = post['id']
    post_title = post.find_all('h2')[0].get_text()
    post_date = dateparser.parse(' '.join(post_title.split()[1::]))
    
    forecasts = post.find_all(id = 'surf-weerbericht')[0]
    forecast_dates, forecast_contents, days_of_week = forecast_parser(forecasts)
    
    df = pd.DataFrame({'report_date': forecast_dates,
                       'report_content': forecast_contents,
                       'day_of_week': days_of_week})

    wetsuit_adviezen_str = wetsuit_recommendation(forecasts)

    df['post_date'] = post_date  # year need to be udpated
    df['post_id'] = post_id
    df['post_title'] = post_title
    df['wetsuit_adviezen_str'] = wetsuit_adviezen_str
    df['wetsuit_adviezen'] = df['day_of_week']

    try:
#         wetsuit_adviezen_str = wetsuit_adviezen_str.get()
        wetsuit_adviezen_dict = str_to_dict(wetsuit_adviezen_str)
        df.replace({"wetsuit_adviezen": wetsuit_adviezen_dict}, inplace=True)

    except:
        df['wetsuit_adviezen'] = None

    return df

def wetsuit_recommendation(forecasts):
    """
    get wetsuit recommendation
    """
    wetsuit_adviezen = None
    
    for content in forecasts.find_all('p'):

        text = re.sub(r'\W+', ' ', content.get_text()).lower()
        key_words = ['wetsuit adviezen', 'Wetsuit advies', "Wetsuit mm advies", 'Wetsuit diktes']

        if any(word.lower() in text for word in key_words):
#             print('a match')
            wetsuit_adviezen = content
            return wetsuit_adviezen.get_text()
        
    return wetsuit_adviezen


def str_to_dict(wetsuit_adviezen):
    """
    convert wetsuit_adviezen to a dictionary that uses dayofweek as the key
    """

    # ['Tests run: 1', ' Failures: 0', ' Errors: 0']
    a = wetsuit_adviezen.split('\n')

    d = {}

    for b in a:
        try:
            i = b.split(': ')
            d[i[0].lower()] = i[1]
        except:
            d[i[0].lower()] = None

    return d

def get_href(url):
    """
    
    """
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    hrefs = soup.find_all('a', text=re.compile('Surfweer*'), rel = 'bookmark')
    return hrefs

def get_post(post_url):
    """
    get 
    """
    page = requests.get(post_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    post = soup.find('div', class_ = "post")
    
    return post

## test replace with dict 

 correct the meer info issue  (some posts were not completely shown)
- to resolve this issue, i will first read the hyper-link 
- use the hyper-link to retrieve the information 


In [4]:
post_urls = []
page_numbers = np.arange(11) + 1

for page_number in page_numbers:
    
    page_url = 'http://surfweer.nl/surf/page/{:}/'.format(page_number)
    hrefs = get_href(page_url)
    post_urls.append([href.get('href') for href in hrefs])
    
post_urls = sum(post_urls, [])

In [5]:
dfs = []

for post_url in post_urls:
    post = get_post(post_url) 
    df = post_to_df(post) 
    df['post_url'] = post_url
    dfs.append(df)

In [6]:
df_combined = pd.concat(dfs)

In [7]:
df_combined

,report_date,report_content,day_of_week,post_date,post_id,post_title,wetsuit_adviezen_str,wetsuit_adviezen,post_url
0,2021-01-12,Dinsdag 12 januari\nHet blijft de hele dag waa...,dinsdag,2021-01-12,post-33062,Surfweer dinsdag 12 januari 2021\n\n,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...","6/5/4mm wetsuit, hand 4mm, schoen 7mm, cap 3mm",http://surfweer.nl/surf/surfweer-dinsdag-12-ja...
1,2021-01-13,Woensdag 13 januari\nIneens zijn de golven twe...,woensdag,2021-01-12,post-33062,Surfweer dinsdag 12 januari 2021\n\n,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...","Nieuwe 5/4mm wetsuit, schoen 6mm en hand 4mm, ...",http://surfweer.nl/surf/surfweer-dinsdag-12-ja...
2,2021-01-14,"Donderdag 14 januari\nDe wind is weg, maar de ...",donderdag,2021-01-12,post-33062,Surfweer dinsdag 12 januari 2021\n\n,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...","wetsuit 6mm of 5/4mm cap +1mm, hand 5mm, schoe...",http://surfweer.nl/surf/surfweer-dinsdag-12-ja...
3,NaT,Vrijdag en het weekend\nWaar de swell donderda...,vrijdag,2021-01-12,post-33062,Surfweer dinsdag 12 januari 2021\n\n,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...",vrijdag,http://surfweer.nl/surf/surfweer-dinsdag-12-ja...
4,NaT,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...","wetsuit-adviezen,",2021-01-12,post-33062,Surfweer dinsdag 12 januari 2021\n\n,"Wetsuit-adviezen, De zee is +6 C\nDinsdag: 6/5...","wetsuit-adviezen,",http://surfweer.nl/surf/surfweer-dinsdag-12-ja...
...,...,...,...,...,...,...,...,...,...
6,NaT,"Wetsuit diktes:\nMaandag: 6/5 of 5/4 wetsuit, ...",wetsuit,2018-12-17,post-31217,Surfweer Maandag 17 december 2018\n\n,"Wetsuit diktes:\nMaandag: 6/5 of 5/4 wetsuit, ...",wetsuit,http://surfweer.nl/surf/surfweer-maandag-17-de...
0,2021-12-11,Dinsdag 11 december\nIn de vroege ochtend is d...,dinsdag,2018-12-11,post-31178,Surfweer dinsdag 11 december 2018\n\n,Wetsuit diktes:\nDinsdag: 5mm met 3mm cap en 5...,5mm met 3mm cap en 5mm schoen + 2mm hand,http://surfweer.nl/surf/surfweer-dinsdag-11-de...
1,2021-12-12,Woensdag 12 december\nIs er nog wat van de swe...,woensdag,2018-12-11,post-31178,Surfweer dinsdag 11 december 2018\n\n,Wetsuit diktes:\nDinsdag: 5mm met 3mm cap en 5...,"6 of 5/4mm met cap, schoen 6mm en hand 3mm",http://surfweer.nl/surf/surfweer-dinsdag-11-de...
2,NaT,De dagen daarna..\nDonderdag is het compleet f...,de,2018-12-11,post-31178,Surfweer dinsdag 11 december 2018\n\n,Wetsuit diktes:\nDinsdag: 5mm met 3mm cap en 5...,de,http://surfweer.nl/surf/surfweer-dinsdag-11-de...


In [16]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2021-01-17


In [18]:
df_combined.to_csv('surfweer_data_{:}.csv'.format(str(today).replace('-','_')))